A un avatar, on associe la langue qui restera la même à chaque requête avec cet avatar. En revanche, la date, le mobile, la ville peuvent changer d'une requete à l'autre. 

La plupart du temps, la ville de recherche est liée à l'avatar, dans un premier, on la fixe carrément à l'avatar.

- Rajouter une variable qui compte le nombre de fois ou un avatar effectue une requete

In [24]:
import urllib.parse
import numpy as np
import requests
import pandas as pd
from csv import writer
from csv import DictWriter
import csv

domain = "51.91.251.0"
port = 3000
host = f"http://{domain}:{port}"
path = lambda x: urllib.parse.urljoin(host, x)

user_id = '5db3e3d5-ee3d-4905-987d-27315733c1f6'

In [25]:
language = ['austrian', 'belgian', 'bulgarian', 'croatian', 'cypriot', 'czech', 'danish', 'dutch', 'estonian', 'finnish',
            'french', 'german', 'greek', 'hungarian', 'irish', 'italian', 'latvian', 'lithuanian', 'luxembourgish', 
            'maltese', 'polish', 'portuguese', 'romanian', 'slovakian', 'slovene', 'spanish','swedish']
city = ['amsterdam', 'copenhagen', 'madrid', 'paris', 'rome', 'sofia', 'valletta', 'vienna', 'vilnius']
mobile = ['0','1']
date = np.arange(0,45,1)
#date = np.arange(0,10,1)

In [26]:
def creation_avatar(N_avatar):
    L = np.random.choice(language)#replace ? ie avec ou sans remplacement
    C = np.random.choice(city)
    A = 'AliBaba-'+str(N_avatar)
    r = requests.post(path(f"avatars/{user_id}/{A}"))
    N_avatar += 1
    return [A,L,C,N_avatar]

In [27]:
def verif_avatar(random_avatar, random_date, tab_old_param):
    good_avatar = True
    for i in tab_old_param.index:
        if (random_avatar==tab_old_param['avatar_name'][i]):
            if (random_date>=tab_old_param['date'][i]): #MODIF
                good_avatar = False
        if good_avatar ==False : break
    return good_avatar

In [28]:
def unicite(new_params, tab_old_param):
    unique = True
    for i in tab_old_param.index:
        if ((new_params['language']==tab_old_param['language'][i]) and 
            (new_params['city']==tab_old_param['city'][i]) and 
            (new_params['date']==tab_old_param['date'][i])):# and
            #(new_params['mobile']==tab_old_params['mobile'][i])): #MODIF
            unique = False
        if unique == False : break
    return unique

In [35]:
# A MODIFIER POUR QUE CE SOIT AUTOMATIQUE
order_requests = 2790 #UPDATE A CHAQUE FOIS dernier numero
N_avatar = 1909 #UPDATE A CHAQUE FOIS numero suivant

for i in range(100):

    #Creation avatar
    new_avatar = creation_avatar(N_avatar)
    N_avatar = new_avatar[3]
    nb_requete = 0

    #D = np.random.choice(date)
    D = 10

    #A COMMENTER SI UNE DATE FIXE
   # while (verif_avatar(new_avatar[0], D, tab_old_param = pd.read_csv('./tab_params_utilises.csv')) & (nb_requete<2)):

    M = np.random.choice(mobile)
    new_params = {
        "avatar_name": new_avatar[0],
        "language": new_avatar[1],
        "city": new_avatar[2],
        "date": D,
        "mobile": M,
            }

    #if unicite:
    if(unicite(new_params, tab_old_param = pd.read_csv('./tab_params_utilises.csv'))):

        nb_requete += 1

        requete = requests.get(path(f"pricing/{user_id}"), params=new_params)

        #stockage des nouveaux paramètres utilisés
        params_requests = []
        params_requests.append(new_params)    
        params_requests  = pd.DataFrame(params_requests)
        old_param = pd.read_csv('./tab_params_utilises.csv')

        new_param_file = pd.concat([old_param,params_requests])
        new_param_file.to_csv("./tab_params_utilises.csv", index = False)


        #stockage des nouvelles requêtes effectuées
        pricing_requests=[]
        pricing_requests.append(pd.DataFrame(requete.json()['prices']).assign(**requete.json()['request']))
        pricing_requests=pd.concat(pricing_requests)

        #Ajout de la colonne nb_requete
        vect2 = np.ones(len(pricing_requests), dtype=int)*nb_requete
        vect2 = pd.DataFrame(vect2,columns = ['nb_requete'])
        pricing_requests = vect2.join(pricing_requests, on=None, how='right', lsuffix='', rsuffix='', sort=False)

        #Ajout de la colonne order_request
        order_requests += 1
        vect = np.ones(len(pricing_requests), dtype=int)*order_requests
        vect = pd.DataFrame(vect,columns = ['order_requests'])
        pricing_requests = vect.join(pricing_requests, on=None, how='right', lsuffix='', rsuffix='', sort=False)

        hotels = pd.read_csv('./defi-ia-2023/features_hotels.csv', index_col=['hotel_id', 'city'])
        pricing_requests = pricing_requests.join(hotels, on=['hotel_id', 'city'])

        df = pd.read_csv('./result_requetes.csv')
        df= pd.concat([df,pricing_requests],axis = 0)
        df.to_csv('./result_requetes.csv', index= False)

        #A COMMENTER SI UNE DATE FIXE
        #D = np.random.choice(date)
              

In [33]:
requete.json()

{'detail': 'A pricing request for this avatar already exists for a sooner date'}